In [1]:
import numpy as np
from tkinter import *
from enum import Enum
from time import sleep

In [2]:
T = 15 # time limit
map_width = 6
map_height = 5
map_size = (map_width, map_height)
our_pos_zero = (0,0)
his_pos_zero = (4,4)
exit_pos = (4,4)

In [3]:
class Step(Enum):
    NONE = (0,0)
    LEFT = (-1,0)
    UP = (0,-1)
    RIGHT = (1,0)
    DOWN = (0,1)

In [4]:
class Direction(Enum):
    NONE = 0
    LEFT = 1
    UP = 2
    RIGHT = 3
    DOWN = 4
    
    @classmethod
    def inverse(cls, direction):
        if direction is cls.NONE:
            return cls.NONE
        if direction is cls.LEFT:
            return cls.RIGHT
        if direction is cls.UP:
            return cls.DOWN
        if direction is cls.RIGHT:
            return cls.LEFT
        if direction is cls.DOWN:
            return cls.UP

In [5]:
def in_map(pos):
    if (pos[0] < 0 or pos[1] < 0 or pos[0] >= map_width or pos[1] >= map_height):
        return False
    return True

In [6]:
# Our partial state (without the minotaur)
class PartialState:
    
    def __init__(self, pos):
        self.pos = pos # (x, y) tuple
        
    def getActions(self):
        walls_dirs = walls.get(self.pos, [])
        return [item for it in Direction if it not in walls_dirs] # add the borders-check
    
    def act(self, direction):
        # Validity check
        if direction not in getActions():
            raise Exception("Illegal action!")
        self.pos = PartialState.step(self.pos, direction)
    
    @classmethod
    def step(cls, pos, direction):
        if direction is Direction.NONE:
            return (pos[0], pos[1])
        if direction is Direction.LEFT:
            return (pos[0]-1, pos[1])
        if direction is Direction.UP:
            return (pos[0], pos[1]-1)
        if direction is Direction.RIGHT:
            return (pos[0]+1, pos[1])
        return (pos[0], pos[1]+1)
    
    @classmethod
    def getActions(cls, pos, walls):
        walls_dirs = walls.get(pos, [])
        return [dr for dr in Direction if dr not in walls_dirs and in_map(PartialState.step(pos, dr))]
    

In [7]:
class State:
    
    def __init__(self, part_state, minotaur_pos):
        self.minotaur_pos = minotaur_pos # (x, y) tuple of the Minotaur's position
        self.part_state = part_state
    


## Defining walls

In [8]:
# One-way walls
walls = { (1,0): [Direction.RIGHT], (1,1): [Direction.RIGHT], (1,2): [Direction.RIGHT],
        (3,1): [Direction.RIGHT], (3,2): [Direction.RIGHT],
        (4,1): [Direction.DOWN], (5,1): [Direction.DOWN],
        (1,3): [Direction.DOWN], (2,3): [Direction.DOWN], (3,3): [Direction.DOWN], (4,3): [Direction.DOWN],
        (3,4): [Direction.RIGHT] }

# New values are appended to the first (base) dictionary
def append_to_dict(base_dict, new_vals):
    for key,vals in new_vals.items():
        base_vals = base_dict.get(key, None)
        if base_vals is None:
            base_vals = []
            base_dict[key] = base_vals
        base_vals.extend(vals)

# Mirroring the walls
mirror_walls = {}
for key,vals in walls.items():
    for val in vals:
        mirror_pos = PartialState.step(key, val)
        mirror_vals = mirror_walls.get(mirror_pos, None)
        if mirror_vals is None:
            mirror_vals = []
            mirror_walls[mirror_pos] = mirror_vals
        mirror_vals.append(Direction.inverse(val))
append_to_dict(walls, mirror_walls)

## Main algorithm

In [9]:
time_left = 30
state_space_size = (time_left, *map_size, *map_size)
V = np.zeros(state_space_size)
theta = 1e-6
delta = 1
it_count = 0

In [10]:
while delta > theta:
    delta = 0.0
    it_count += 1
    for t_left, our_x, our_y, his_x, his_y in np.ndindex(state_space_size):
        our_pos = (our_x, our_y)
        his_pos = (his_x, his_y)
        s = (t_left, *our_pos, *his_pos)
        v = V[s]
        if t_left == 0 and our_pos != exit_pos:
            V[s] = -1 # Ran out of time
        elif our_pos == his_pos:
            V[s] = -1 # Eaten
        elif our_pos == exit_pos:
            V[s] = 1 # Made it
        else:
            his_pos_new = [PartialState.step(his_pos, act) for act in PartialState.getActions(his_pos, walls)]
            p = 1/len(his_pos_new)
            best_act_value = -np.infty
            for direction in PartialState.getActions(our_pos, walls):
                our_pos_new = PartialState.step(our_pos, direction)
                new_val = np.sum([V[(t_left-1, *our_pos_new, *his_p)] for his_p in his_pos_new]) * p
                if (new_val > best_act_value):
                    best_act_value = new_val
            V[s] = best_act_value
        new_delta = np.abs(v - V[s])
        if new_delta > delta:
            delta = new_delta

## Visualization

In [41]:
class Cell():
    FILLED_COLOR_BG = "green"
    EMPTY_COLOR_BG = "white"
    FILLED_COLOR_BORDER = "green"
    EMPTY_COLOR_BORDER = "black"

    def __init__(self, master, x, y, size, fill="white", outline="black"):
        """ Constructor of the object called by Cell(...) """
        self.master = master
        self.abs = x
        self.ord = y
        self.size= size
        self.fill= fill
        self.outline=outline

    def _switch(self):
        """ Switch if the cell is filled or not. """
        self.fill= not self.fill

    def draw(self):
        """ order to the cell to draw its representation on the canvas """
        if self.master != None :
            fill = self.fill
            outline = self.outline
            
            xmin = self.abs * self.size
            xmax = xmin + self.size
            ymin = self.ord * self.size
            ymax = ymin + self.size

            self.master.create_rectangle(xmin, ymin, xmax, ymax, fill = fill, outline = outline)

class CellGrid(Canvas):
    def __init__(self, master, rowNumber, columnNumber, cellSize, our_position, his_position, walls, time_left, *args, **kwargs):
        Canvas.__init__(self, master, width = cellSize * columnNumber , height = cellSize * rowNumber, *args, **kwargs)

        self.cellSize = cellSize
        self.grid = []
        for row in range(rowNumber):
            line = []
            for column in range(columnNumber):
                line.append(Cell(self, column, row, cellSize))
            self.grid.append(line)
        
        self.our_position = our_position
        self.his_position = his_position
        self.walls = walls
        self.time_left = time_left
        
        #memorize the cells that have been modified to avoid many switching of state during mouse motion.
        #self.switched = []

        #bind click action
        self.bind("<Button-1>", self.handleMouseClickStep)  
        #bind moving while clicking
        #self.bind("<B1-Motion>", self.handleMouseMotion)
        #bind release button action - clear the memory of midified cells.
        #self.bind("<ButtonRelease-1>", lambda event: self.switched.clear())

        self.draw()
        self.finished = False
        

    def draw(self):
        for row in self.grid:
            for cell in row:
                cell.draw()
        # Drawing the players
        Cell(self, self.our_position[0], self.our_position[1], self.cellSize, "green", Cell.FILLED_COLOR_BORDER).draw()
        Cell(self, self.his_position[0], self.his_position[1], self.cellSize, "red", Cell.FILLED_COLOR_BORDER).draw()
        
        
        

    def _eventCoords(self, event):
        row = int(event.y / self.cellSize)
        column = int(event.x / self.cellSize)
        return row, column

    def handleMouseClickStep(self, event):
        if not self.finished:
            self.step()
        
        
    def handleMouseClick(self, event):
        row, column = self._eventCoords(event)
        cell = self.grid[row][column]
        cell._switch()
        cell.draw()
        #add the cell to the list of cell switched during the click
        self.switched.append(cell)

    def handleMouseMotion(self, event):
        row, column = self._eventCoords(event)
        cell = self.grid[row][column]

        if cell not in self.switched:
            cell._switch()
            cell.draw()
            self.switched.append(cell)
    
    def step(self):
        #sleep(0.3)
        best_pos = None
        best_val = -np.infty
        #time_left = time_horizon-t
        his_pos_new = [PartialState.step(self.his_position, act) for act in PartialState.getActions(self.his_position, walls)]
        his_p_size = len(his_pos_new)
        for direction in PartialState.getActions(self.our_position, self.walls):
            our_pos_new = PartialState.step(self.our_position, direction)
            val = np.sum([V[(self.time_left-1, *our_pos_new, *his_p_new)] for his_p_new in his_pos_new])/his_p_size
            if val > best_val:
                best_val = val
                best_pos = our_pos_new
        self.our_position = best_pos
        self.time_left = self.time_left-1

        self.his_position = his_pos_new[np.random.choice(his_p_size)]
        
        self.draw()
        if self.our_position == self.his_position or self.time_left == 0:
            self.finished = True
            
app = Tk()
grid = CellGrid(app, map_height, map_width, 50, our_pos_zero, his_pos_zero, walls, T)
grid.pack()
app.mainloop()

In [16]:
np.random.choice(5)

0

In [22]:
x_range = np.arange(map_width)
y_range = np.arange(map_height)
mixed_states = np.meshgrid(x_range, y_range)
print(mixed_states)

[array([[0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5]]), array([[0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4]])]


In [9]:
pos = (0,1)
PartialState.step(pos, Direction.DOWN)

(0, 2)